In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
def generate_dataset(amount, length, freedom):
    x = np.random.rand(amount,length)
    y = np.sum(x[:,:freedom],axis=1)
    y.shape=(100,1)
    return np.asarray(x,dtype=np.float32),np.asarray(y,dtype=np.float32)

In [ ]:
input_dataset,output_dataset = generate_dataset(100,10000,10)
print(input_dataset.shape,output_dataset.shape)

In [ ]:
# class DeconvolutionDataset(Dataset):
#     """dataset."""

#     def __init__(self, input_dataset,output_dataset, transform=None):
#         """

#         """
#         self.input_dataset = input_dataset
#         self.output_dataset = output_dataset
#         self.transform = transform

#     def __len__(self):
#         return len(self.output_dataset)

#     def __getitem__(self, idx):

#         sample = {'input': self.input_dataset[idx,:], 'output': self.output_dataset[idx,:]}
        
#         if self.transform:
#             sample = self.transform(sample)

#         return sample


# # In[4]:


# class ToTensor(object):
#     """Convert ndarrays in sample to Tensors."""

#     def __call__(self, sample):
#         input_image, output_image = sample['input'], sample['output']
#         return {'input': torch.from_numpy(input_image),
#                 'output': torch.from_numpy(output_image)}


In [ ]:
# training_dataset = DeconvolutionDataset(input_dataset,output_dataset,transform=ToTensor())
# print(training_dataset)
# dataloader = DataLoader(training_dataset, batch_size=1,shuffle=True, num_workers=4)
# for batch_idx, sample in enumerate(dataloader):
#     print(sample['input'].size(),sample['output'].size())

In [ ]:
import torch
from torch import nn, optim
from torch.autograd import Variable

In [ ]:
# Linear Regression Model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)  # input and output is 1 dimension
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(100,1)

    def forward(self, x):
        out_layer1 = self.relu(self.linear1(x))
        out = self.linear2(out_layer1)
        return out

In [ ]:
device = torch.device("cuda:0")
model = LinearRegression()
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
x_train = torch.from_numpy(input_dataset)
y_train = torch.from_numpy(output_dataset)

In [ ]:
num_epochs = 10000
for epoch in range(num_epochs):
    inputs = Variable(x_train).cuda()
    target = Variable(y_train).cuda()
    
    # forward
    out = model(inputs)
    loss = criterion(out, target)
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
#     for batch_idx, sample in enumerate(dataloader):
#         inputs,target = sample['input'].to(device),sample['output'].to(device)

#         # forward
#         out = model(inputs)
#         loss = criterion(out, target)
#         # backward
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

    if (epoch+1) % 20 == 0:
        print('Epoch[{}/{}], loss: {:.6f}'
              .format(epoch+1, num_epochs, loss.data[0]))


In [ ]:
model.eval()
predict = model(Variable(x_train).cuda())
predict = predict.data.cpu().numpy()

In [ ]:
plt.plot(output_dataset)

In [ ]:
plt.plot(predict)